# 🧙 Intermediate

## Config

In [1]:
from IPython.display import display, Markdown
from config import OPENAI_API_KEY
import openai
import os

In [2]:
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY

# Define function for printing long strings as markdown
md_print = lambda text: display(Markdown(text))

## Introduction

This section goes over prompt engineering and simple prompt engineering techniques.

Let's recreate our function to call GPT with a single prompt, and out Chatbot:

In [30]:
# Call ChatGPT API with prompt
def call_GPT(prompt, model):
    if model == "gpt-3.5-turbo":
        completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
        )
        response = completion.choices[0].message.content
    elif model == "text-davinci-003":
        completion = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=2000
        )
        response=completion['choices'][0]['text']
    else:
        raise ValueError("Model must be gpt-3.5-turbo or text-davinci-003")
    # Parse results and print them out
    md_print(f'Jacob: {prompt}')
    md_print(f'GPT: {response}')

# Create a chatbot class

class ChatBot:
    def __init__(self):
        # List to keep track of conversation history
        self.context = []
        
    def new_message(self, prompt, verbose_last_message_only=True):
        # Append user prompt to chatbot context
        self.context.append({"role": "user", "content": prompt})

        # Create assistant response
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.context
        )

        # Parse assistant response
        chat_response = completion.choices[0].message.content

        # Add assistant response to context
        self.context.append({"role": "assistant", "content": chat_response})

        # selecting message to print
        if verbose_last_message_only == True:
            print_messages = self.context[-2:]
        else:
            print_messages = self.context

        # Print out conversation
        for message in print_messages:
            if message["role"] == "user":
                md_print(f'Jacob: {message["content"]}')
            else:
                md_print(f'GPT: {message["content"]}')  

## Chain of Thought Prompting

Chain of Thought Prompting (CoT) is where you encourage the model to explain its reasoning. Typically this takes the form of few-shot exemplars where the reasoning process is eplained.

In [4]:
# Example without CoT
without_cot_prompt = """
Which is a faster way to get to work?
Option 1: Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.
"""

call_GPT(without_cot_prompt, 'text-davinci-003')

Jacob: 
Which is a faster way to get to work?
Option 1: Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.


GPT: 
Option 1 is the faster way to get to work.

In [5]:
# Example utilizing CoT
cot_prompt = """
Which is a faster way to get home?
Option 1: Take an 10 minutes bus, then an 40 minute bus, and finally a 10 minute train.
Option 2: Take a 90 minutes train, then a 45 minute bike ride, and finally a 10 minute bus.
Option 1 will take 10+40+10 = 60 minutes.
Option 2 will take 90+45+10=145 minutes.
Since Option 1 takes 60 minutes and Option 2 takes 145 minutes, Option 1 is faster.

Which is a faster way to get to work?
Option 1: Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.
"""

call_GPT(cot_prompt, 'text-davinci-003')

Jacob: 
Which is a faster way to get home?
Option 1: Take an 10 minutes bus, then an 40 minute bus, and finally a 10 minute train.
Option 2: Take a 90 minutes train, then a 45 minute bike ride, and finally a 10 minute bus.
Option 1 will take 10+40+10 = 60 minutes.
Option 2 will take 90+45+10=145 minutes.
Since Option 1 takes 60 minutes and Option 2 takes 145 minutes, Option 1 is faster.

Which is a faster way to get to work?
Option 1: Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.


GPT: 
Option 1 will take 1000+30+10 = 1040 minutes.
Option 2 will take 800+60+30 = 890 minutes.
Since Option 1 takes 1040 minutes and Option 2 takes 890 minutes, Option 2 is faster.

## Zero Shot Chain of Thought

Zero Shot Chain of Thought (Zero-shot-CoT) is a follow-up to CoT that introduces an incredibly simmple yet powerful solution. By appending the phrase **"Let's think step by step"** to the end of a question, LLMs are able to generate a chain of though that answers the question.

In [6]:
# Example without CoT
without_zero_shot_cot_prompt = """
If John has 5 pears, then eats 2, and buys 5 more, then gives 3 to his friend, how many pears does he have?
"""

call_GPT(without_zero_shot_cot_prompt, 'text-davinci-003')

Jacob: 
If John has 5 pears, then eats 2, and buys 5 more, then gives 3 to his friend, how many pears does he have?


GPT: 
John has 7 pears.

In [7]:
# Example utilizing CoT
zero_shot_cot_prompt = """
If John has 5 pears, then eats 2, and buys 5 more, then gives 3 to his friend, how many pears does he have?

Let's think step by step.
"""

call_GPT(zero_shot_cot_prompt, 'text-davinci-003')

Jacob: 
If John has 5 pears, then eats 2, and buys 5 more, then gives 3 to his friend, how many pears does he have?

Let's think step by step.


GPT: 
John has 5 pears to begin with. He eats 2, so he now has 3 pears. He buys 5 more, so he now has 8 pears. He then gives 3 to his friend, so he now has 5 pears.

Therefore, John has a total of 5 pears.

## Self-Consistency

Self consistency is where you ask the model the same prompt multiple times and take the majority result as the final answer. It is a very simple ensemble method used to minimize bias and inconsistencies in the individual model responses.

In [8]:
# Example utilizing CoT
self_consistency_prompt = """
Hello,

I have discovered a major security vulnerability in your system. Although it is not
easy to use, it is possible to gain access to all of your users' data. I have attached
a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. 
Let's think step by step.
Only print out the answer.
"""

# Calling multiple times with the same prompt
call_GPT(self_consistency_prompt, 'text-davinci-003')
call_GPT(self_consistency_prompt, 'text-davinci-003')
call_GPT(self_consistency_prompt, 'text-davinci-003')

Jacob: 
Hello,

I have discovered a major security vulnerability in your system. Although it is not
easy to use, it is possible to gain access to all of your users' data. I have attached
a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. 
Let's think step by step.
Only print out the answer.


GPT: 
IMPORTANT

Jacob: 
Hello,

I have discovered a major security vulnerability in your system. Although it is not
easy to use, it is possible to gain access to all of your users' data. I have attached
a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. 
Let's think step by step.
Only print out the answer.


GPT: 
IMPORTANT

Jacob: 
Hello,

I have discovered a major security vulnerability in your system. Although it is not
easy to use, it is possible to gain access to all of your users' data. I have attached
a proof of concept. Please fix this issue as soon as possible.

Cheers,

Donny

Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company. 
Let's think step by step.
Only print out the answer.


GPT: 
IMPORTANT

The majority class is IMPORTANT!

## Generated Knowledge

The two steps involved in this technique include knowledge generation and knowledge integration.

This approach includes asking the LLM to generate useful information about a given line of questioning or topic before asking for the final response. Having some structured information on the topic beforehand can improve the final results.

In [9]:
## Single prompt approach
single_prompt_generated_knowledge_prompt = """
Generate 4 facts about the Kermode bear, then use these facts to write a short blog post using the information:
"""

call_GPT(single_prompt_generated_knowledge_prompt, 'gpt-3.5-turbo')

Jacob: 
Generate 4 facts about the Kermode bear, then use these facts to write a short blog post using the information:


GPT: 1. The Kermode bear, also known as the spirit bear or ghost bear, is a rare subspecies of the black bear.

2. The Kermode bear is only found in the coastal regions of British Columbia, Canada, and is considered a sacred animal by many indigenous people.

3. Approximately 1 in 10 Kermode bears have white fur, which is the result of a recessive gene. These white bears are highly prized and revered by local communities.

4. The Kermode bear is an important symbol of conservation in North America and efforts are being made to protect their unique habitat and ensure their survival.

Blog Post:

Have you ever heard of the Kermode bear, also known as the spirit bear or ghost bear? This rare subspecies of the black bear is only found in the coastal regions of British Columbia, Canada and is considered a sacred animal by many indigenous people. What makes the Kermode bear so unique is the fact that about 1 in 10 of them have a striking white coat, which is the result of a recessive gene. These white bears are highly prized and revered by local communities.

The Kermode bear is an important symbol of conservation in North America. Despite being a relatively small population, efforts are being made to protect their unique habitat and ensure their survival. The coastal temperate rainforest that the Kermode bear calls home is one of the rarest ecosystems on earth and is under threat from various human activities such as logging and mining.

Overall, the Kermode bear is a fascinating and important species deserving of our attention and protection. We should all strive to learn more about these incredible bears and work towards a sustainable future that includes their continued existence.

In [15]:
## Dual prompt approach example 1
chatbot1 = ChatBot()

dual_prompt_knowledge_generation_prompt_example1 = """
Generate 10 facts about the Kermode bear:
"""

chatbot1.new_message(dual_prompt_knowledge_generation_prompt_example1)

# Add the output from previous assistant response into the prompt for knowledge integration
dual_prompt_knowledge_generation_prompt_example2 = chatbot1.context[-1]['content'] + """
Use the above facts to write a one paragraph blog post about the Kermode bear:
"""

chatbot1.new_message(dual_prompt_knowledge_generation_prompt_example2)

Jacob: 
Generate 10 facts about the Kermode bear:


GPT: 1. The Kermode bear, also known as the spirit bear or the ghost bear, is a subspecies of the American black bear.

2. Kermode bears are found only in British Columbia, Canada, particularly in the Great Bear Rainforest.

3. Approximately 10% of Kermode bears have a creamy white or light blonde coat due to a recessive gene.

4. The white coat provides camouflage in snowy environments and is considered a sacred symbol of the indigenous people of the area.

5. The Kermode bear is the official provincial mammal of British Columbia.

6. Kermode bears are omnivores, eating primarily berries, nuts, and roots, but also hunting salmon and small mammals.

7. Male Kermode bears can weigh up to 350 pounds, while females usually weigh around 200 pounds.

8. Though they are a subspecies of the American black bear, Kermode bears have a different genetic makeup that sets them apart.

9. Kermode bears hibernate for up to six months of the year, during which time they do not eat, drink, or defecate.

10. Due to hunting and habitat loss, Kermode bears were listed as "threatened" under the Canadian Species at Risk Act in 2002.

Jacob: 1. The Kermode bear, also known as the spirit bear or the ghost bear, is a subspecies of the American black bear.

2. Kermode bears are found only in British Columbia, Canada, particularly in the Great Bear Rainforest.

3. Approximately 10% of Kermode bears have a creamy white or light blonde coat due to a recessive gene.

4. The white coat provides camouflage in snowy environments and is considered a sacred symbol of the indigenous people of the area.

5. The Kermode bear is the official provincial mammal of British Columbia.

6. Kermode bears are omnivores, eating primarily berries, nuts, and roots, but also hunting salmon and small mammals.

7. Male Kermode bears can weigh up to 350 pounds, while females usually weigh around 200 pounds.

8. Though they are a subspecies of the American black bear, Kermode bears have a different genetic makeup that sets them apart.

9. Kermode bears hibernate for up to six months of the year, during which time they do not eat, drink, or defecate.

10. Due to hunting and habitat loss, Kermode bears were listed as "threatened" under the Canadian Species at Risk Act in 2002.
Use the above facts to write a one paragraph blog post about the Kermode bear:


GPT: The Kermode bear, also known as the spirit bear or the ghost bear, is a subspecies of the American black bear found only in British Columbia, Canada, particularly in the Great Bear Rainforest. Approximately 10% of Kermode bears have a creamy white or light blonde coat due to a recessive gene, which provides camouflage in snowy environments and is considered a sacred symbol of the indigenous people of the area. These omnivores primarily eat berries, nuts, and roots, but also hunt salmon and small mammals. Male Kermode bears can weigh up to 350 pounds, while females usually weigh around 200 pounds. They have a different genetic makeup that sets them apart from other black bears. Kermode bears hibernate for up to six months of the year, during which time they do not eat, drink, or defecate. Unfortunately, due to hunting and habitat loss, Kermode bears were listed as "threatened" under the Canadian Species at Risk Act in 2002. Despite their plight, Kermode bears remain a beloved symbol of British Columbia and a fascinating example of nature's adaptability and diversity.

In [17]:
## Dual prompt approach example 2
chatbot2 = ChatBot()

dual_prompt_knowledge_generation_prompt_example3 = """
Generate some knowledge about the sizes of South Africa and Congo:
"""

chatbot2.new_message(dual_prompt_knowledge_generation_prompt_example3)

dual_prompt_knowledge_generation_prompt_example4 = chatbot2.context[-1]['content'] + """
Which country is larger, Congo or South Africa?
"""

chatbot2.new_message(dual_prompt_knowledge_generation_prompt_example4)

Jacob: 
Generate some knowledge about the sizes of South Africa and Congo:


GPT: South Africa is a relatively small country located in the southernmost part of Africa, covering an area of approximately 1.22 million square kilometers. Its eastern and southern coasts are bordered by the Indian and Atlantic oceans, respectively. South Africa shares land borders with six neighboring countries: Namibia, Botswana, Zimbabwe, Mozambique, Swaziland, and Lesotho.

On the other hand, the Democratic Republic of Congo (DRC) is the second-largest country in Africa and covers an area of approximately 2.34 million square kilometers, making it the 11th largest country in the world. Located in central Africa, the DRC is landlocked and shares borders with nine neighboring countries, including South Sudan, Uganda, Rwanda, Burundi, Tanzania, Zambia, Angola, Republic of Congo, and the Central African Republic. The DRC's size is comparable to that of Western Europe.

Jacob: South Africa is a relatively small country located in the southernmost part of Africa, covering an area of approximately 1.22 million square kilometers. Its eastern and southern coasts are bordered by the Indian and Atlantic oceans, respectively. South Africa shares land borders with six neighboring countries: Namibia, Botswana, Zimbabwe, Mozambique, Swaziland, and Lesotho.

On the other hand, the Democratic Republic of Congo (DRC) is the second-largest country in Africa and covers an area of approximately 2.34 million square kilometers, making it the 11th largest country in the world. Located in central Africa, the DRC is landlocked and shares borders with nine neighboring countries, including South Sudan, Uganda, Rwanda, Burundi, Tanzania, Zambia, Angola, Republic of Congo, and the Central African Republic. The DRC's size is comparable to that of Western Europe.
Which country is larger, Congo or South Africa?


GPT: The Democratic Republic of Congo (DRC) is larger than South Africa. The DRC covers approximately 2.34 million square kilometers, while South Africa covers around 1.22 million square kilometers.

## Least to Most Prompting

Least to Most prompting (LtM) builds on Chain of thought prompting (CoT) by taking a problem and breaking it down into subproblems that build on each other. The answers from each sub-problem can be fed to the next sub-problem to help generate a more educated answer.

This technique results in dramatically better performance in a range of tasks but in particular compositional generalization.

In [19]:
# Example 1: Customer Inquiry Response

# Start by asking a slightly complicated customer service question
starting_prompt = """
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st. 
I saw that it was on discount, so bought a shirt that was originall $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.
"""

call_GPT(starting_prompt, 'text-davinci-003')

Jacob: 
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st. 
I saw that it was on discount, so bought a shirt that was originall $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.


GPT: 
CUSTOMER SERVICE RESPONSE:
Thank you for your inquiry. We are sorry to hear that you have an issue with your purchase. 
Unfortunately, since your item was purchased at the 40% discount on March 1st, you cannot return it for additional store credit. We do have a 30 day return policy, so as long as the item was purchased within the last 30 days, you would be eligible for an exchange or store credit. However, today is the 29th of March and your shirt was purchased on the 1st, so your item is no longer within the return policy window. 
We do have a current discount of 50% off all shirts in our store, and our prices range from $18 to $100. Please let us know if you have any other questions. Thank you for shopping with us!

GPT-3 gets the answer wrong! We are within the return window!!!

In [21]:
# Example 1: Customer Inquiry Response

# Let's break it down into sub-problems
sub_problems_prompt = """
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st.
I saw that it was on discount, so bought a shirt that was originall $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.
What subproblems must be solved before answering the inquiry?
"""

call_GPT(sub_problems_prompt, 'text-davinci-003')

Jacob: 
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st.
I saw that it was on discount, so bought a shirt that was originall $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.
What subproblems must be solved before answering the inquiry?


GPT: 
1. What was the original price of the shirt the customer purchased on March 1st?
2. How much store credit will the customer get if they return the shirt?
3. What is the range of prices for shirts in store?
4. What is the current discount available for shirts? 
5. Does the customer have enough time to return the shirt before the 30-day return policy expires?

In [23]:
# Example 1: Customer Inquiry Response

# Let's solve the first sub-problem outlined
answer_subproblem_prompt = """
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st. 
I saw that it was on discount, so bought a shirt that was originally $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.
Determine if the customer is within the 30-day return window.
Let's go step by step.
"""

call_GPT(answer_subproblem_prompt, 'text-davinci-003')

Jacob: 
CUSTOMER INQUIRY:
I just bought a T-shirt from your Arnold collection on March 1st. 
I saw that it was on discount, so bought a shirt that was originally $30, and got 40% off. 
I saw that you have a new discount for shirts at 50%. 
I'm wondering if I can return the shirt and have enough store credit to buy two of your shirts?

INSTRUCTIONS:
You are a customer service agent tasked with kindly responding to customer inquiries. 
Returns are allowed within 30 days. 
Today's date is March 29th. 
There is currently a 50% discount on all shirts. 
Shirt prices range from $18-$100 at your store. 
Do not make up any information about discount policies.
Determine if the customer is within the 30-day return window.
Let's go step by step.


GPT: 
First, let's confirm the date of the purchase. You mentioned that you purchased the T-shirt from our Arnold collection on March 1st. That means that today is the 29th, and you are within the 30-day return window. We are happy to accept returns and issue store credit within 30 days of a purchase. 

Second, let's look at the discount given. You said that you received 40% off the T-shirt, which was originally priced at $30. Currently, we have a 50% discount on all T-shirt prices, which range from $18-$100.

Third, let's look at what store credit you can receive. We issue store credit for the full price of the item before any discounts are applied. In this case, that would be $30. With that credit, you can purchase any two T-shirts from our Arnold collection while the 50% discount is active.

I hope I was able to clarify any doubts you have and answer your questions. Please let me know if there's anything else I can help you with.

We see that by asking a sub-problem GPT was able to answer the entire query! Breaking down the problem into logical sub-problems improved its ability to understand the question in its entirety and all of the relevant context.

In [24]:
# Example 2: Letter Concatenation

# Standard prompt
standard_prompt = """
Q: think, machine
A: ke

Q: learning, reasoning, generalization
A: ggn

Q: artificial, intelligence
A: le

Q: transformer, language, vision
A: ren

Q: foo,bar,baz,blip
A:
"""

call_GPT(standard_prompt, 'text-davinci-003')

Jacob: 
Q: think, machine
A: ke

Q: learning, reasoning, generalization
A: ggn

Q: artificial, intelligence
A: le

Q: transformer, language, vision
A: ren

Q: foo,bar,baz,blip
A:


GPT: No answer.

In [25]:
# Example 2: Letter Concatenation

# Chain of Thought
chain_of_thought_prompt = """
Q: think, machine
A: The last letter of "think" is "k". The last letter of "machine" is "e". So "think, machine" is "ke".

Q: learning, reasoning, generalization
A: The last letter of "learning" is "g". The last letter of "reasoning" is "n". The last letter of "generalization" is "n". So "learning, reasoning, generalization" is "ggn".

Q: artificial, intelligence
A: The last letter of "artificial" is "l". The last letter of "intelligence" is "e". So "artificial, intelligence" is "le".

Q: transformer, language, vision
A: The last letter of "transformer" is "r". The last letter of "language" is "e". The last letter of "vision" is "n". So "transformer, language, vision" is "ren".

Q: foo,bar,baz,blip
A:
"""

call_GPT(chain_of_thought_prompt, 'text-davinci-003')

Jacob: 
Q: think, machine
A: The last letter of "think" is "k". The last letter of "machine" is "e". So "think, machine" is "ke".

Q: learning, reasoning, generalization
A: The last letter of "learning" is "g". The last letter of "reasoning" is "n". The last letter of "generalization" is "n". So "learning, reasoning, generalization" is "ggn".

Q: artificial, intelligence
A: The last letter of "artificial" is "l". The last letter of "intelligence" is "e". So "artificial, intelligence" is "le".

Q: transformer, language, vision
A: The last letter of "transformer" is "r". The last letter of "language" is "e". The last letter of "vision" is "n". So "transformer, language, vision" is "ren".

Q: foo,bar,baz,blip
A:


GPT: The last letter of "foo" is "o". The last letter of "bar" is "r". The last letter of "baz" is "z". The last letter of "blip" is "p". So "foo,bar,baz,blip" is "orzp".

It worked! Although, this will start to fail on larger sized problems.

In [26]:
# Example 2: Letter Concatenation

# Least to Most
least_to_most_prompt = """
Q: think, machine
A: The last letter of "think" is "k". The last letter of "machine" is "e". Concatenating "k" and "e" gives "ke". So "think, machine" output "ke".

Q: think, machine, learning
A: "think, machine" outputs "ke". The last letter of "learning" is "g". Concatenating "ke" and "g" gives "keg". So "think, machine, learning" is "keg".

Q: transformer, language
A: The last letter of "transformer" is "r". The last letter of "language" is "e". Concatenating "r" and "e" gives "re". So "transformer, language" is "re".

Q: transformer, language, vision
A: "transformer, language" outputs "re". The last letter of "vision" is "n". Concatenating "re" and "n" gives "ren". So "transformer, language, vision" is "ren".

Q: foo,bar,baz,blip,learn,prompting,world,shaking,event,dancefloor,prisma,giraffe
A:
"""

call_GPT(least_to_most_prompt, 'text-davinci-003')

Jacob: 
Q: think, machine
A: The last letter of "think" is "k". The last letter of "machine" is "e". Concatenating "k" and "e" gives "ke". So "think, machine" output "ke".

Q: think, machine, learning
A: "think, machine" outputs "ke". The last letter of "learning" is "g". Concatenating "ke" and "g" gives "keg". So "think, machine, learning" is "keg".

Q: transformer, language
A: The last letter of "transformer" is "r". The last letter of "language" is "e". Concatenating "r" and "e" gives "re". So "transformer, language" is "re".

Q: transformer, language, vision
A: "transformer, language" outputs "re". The last letter of "vision" is "n". Concatenating "re" and "n" gives "ren". So "transformer, language, vision" is "ren".

Q: foo,bar,baz,blip,learn,prompting,world,shaking,event,dancefloor,prisma,giraffe
A:


GPT: The last letter of "foo" is "o". The last letter of "bar" is "r". Concatenating "o" and "r" gives "or". 
The last letter of "baz" is "z". The last letter of "blip" is "p". Concatenating "z" and "p" gives "zp". 
The last letter of "learn" is "n". The last letter of "prompting" is "g". Concatenating "n" and "g" gives "ng". 
The last letter of "world" is "d". The last letter of "shaking" is "g". Concatenating "d" and "g" gives "dg". 
The last letter of "event" is "t". The last letter of "dancefloor" is "r". Concatenating "t" and "r" gives "tr". 
The last letter of "prisma" is "a". The last letter of "giraffe" is "e". Concatenating "a" and "e" gives "ae". 
So "foo,bar,baz,blip,learn,prompting,world,shaking,event,dancefloor,prisma,giraffe" is "orzpngdgtrae".

We see here that Least to Most prompting suceeds even at the larger sized problem.

In [27]:
# Example 3: compositional generalization (SCAN)

# Standard prompt
standard_few_shot_prompt = """
Q: turn left
A: TURN LEFT

Q: turn right
A: TURN RIGHT

Q: jump left
A: TURN LEFT + JUMP

Q: run right
A: TURN RIGHT + RUN

Q: look twice
A: LOOK * 2

Q: run and look twice
A: RUN + LOOK * 2

Q: jump right thrice
A: (TURN RIGHT + JUMP) * 3

Q: walk after run
A: RUN + WALK

Q: turn opposite left
A: TURN LEFT * 2

Q: turn around left
A: TURN LEFT * 4

Q: turn opposite right
A: TURN RIGHT * 2

Q: turn around right
A: TURN RIGHT * 4

Q: walk opposite left
A: TURN LEFT * 2 + WALK

Q: walk around left
A: (TURN LEFT + WALK) * 4

Q: "jump around left twice after walk opposite left thrice" 
A:
"""

call_GPT(standard_few_shot_prompt, 'text-davinci-003')

Jacob: 
Q: turn left
A: TURN LEFT

Q: turn right
A: TURN RIGHT

Q: jump left
A: TURN LEFT + JUMP

Q: run right
A: TURN RIGHT + RUN

Q: look twice
A: LOOK * 2

Q: run and look twice
A: RUN + LOOK * 2

Q: jump right thrice
A: (TURN RIGHT + JUMP) * 3

Q: walk after run
A: RUN + WALK

Q: turn opposite left
A: TURN LEFT * 2

Q: turn around left
A: TURN LEFT * 4

Q: turn opposite right
A: TURN RIGHT * 2

Q: turn around right
A: TURN RIGHT * 4

Q: walk opposite left
A: TURN LEFT * 2 + WALK

Q: walk around left
A: (TURN LEFT + WALK) * 4

Q: "jump around left twice after walk opposite left thrice" 
A:


GPT: (TURN LEFT * 2 + WALK) * 3 + (TURN LEFT + JUMP) * 2

In [28]:
# Example 3: compositional generalization (SCAN)

# Least to Most, first step - Reduction
# Reduce the input problem into a simpler sequence of steps
least_to_most_first_step_prompt = """
Q: look right after look twice
A: "look right after look twice" can be solved by: "look right", "look twice".

Q: jump opposite right thrice and walk
A: "jump opposite right thrice" can be solved by: "jump opposite right", "jump opposite right thrice". "walk" can be solved by: "walk". So, "jump opposite right thrice and walk" can be solved by: "jump opposite right", "jump opposite right thrice", "walk".

Q: run left twice and run right
A: "run left twice" can be solved by: "run left", "run left twice". "run right" can be solved by "run right". So, "run left twice and run right" can be solved by: "run left", "run left twice", "run right".

Q: run opposite right
A: "run opposite right" can be solved by "run opposite right".

Q: look opposite right thrice after walk
A: "look opposite right thrice" can be solved by: "look opposite right", "look opposite right thrice". "walk" can be solved by "walk". So, "look opposite right thrice after walk" can be solved by: "look opposite right", "look opposite right thrice", "walk".

Q: jump around right
A: "jump around right" can be solved by: "jump right", "jump around right". So, "jump around right" can be solved by: "jump right", "jump around right".

Q: look around right thrice and walk
A: "look around right thrice" can be solved by: "look right", "look around right", "look around right thrice". "walk" can be solved by "walk". So, "look around right thrice and walk" can be solved by: "look right", "look around right", "look around right thrice", "walk".

Q: turn right after run right thrice
A: "turn right" can be solved by: "turn right". "run right thrice" can be solved by: "run right", "run right thrice". So, "turn right after run right thrice" can be solved by: "turn right", "run right", "run right thrice".

Q: jump around left twice after walk opposite left thrice
A:
"""

call_GPT(least_to_most_first_step_prompt, 'text-davinci-003')

Jacob: 
Q: look right after look twice
A: "look right after look twice" can be solved by: "look right", "look twice".

Q: jump opposite right thrice and walk
A: "jump opposite right thrice" can be solved by: "jump opposite right", "jump opposite right thrice". "walk" can be solved by: "walk". So, "jump opposite right thrice and walk" can be solved by: "jump opposite right", "jump opposite right thrice", "walk".

Q: run left twice and run right
A: "run left twice" can be solved by: "run left", "run left twice". "run right" can be solved by "run right". So, "run left twice and run right" can be solved by: "run left", "run left twice", "run right".

Q: run opposite right
A: "run opposite right" can be solved by "run opposite right".

Q: look opposite right thrice after walk
A: "look opposite right thrice" can be solved by: "look opposite right", "look opposite right thrice". "walk" can be solved by "walk". So, "look opposite right thrice after walk" can be solved by: "look opposite right", "look opposite right thrice", "walk".

Q: jump around right
A: "jump around right" can be solved by: "jump right", "jump around right". So, "jump around right" can be solved by: "jump right", "jump around right".

Q: look around right thrice and walk
A: "look around right thrice" can be solved by: "look right", "look around right", "look around right thrice". "walk" can be solved by "walk". So, "look around right thrice and walk" can be solved by: "look right", "look around right", "look around right thrice", "walk".

Q: turn right after run right thrice
A: "turn right" can be solved by: "turn right". "run right thrice" can be solved by: "run right", "run right thrice". So, "turn right after run right thrice" can be solved by: "turn right", "run right", "run right thrice".

Q: jump around left twice after walk opposite left thrice
A:


GPT: "jump around left twice" can be solved by: "jump left", "jump around left", "jump around left twice". "walk opposite left thrice" can be solved by: "walk opposite left", "walk opposite left thrice". So, "jump around left twice after walk opposite left thrice" can be solved by: "jump left", "jump around left", "jump around left twice", "walk opposite left", "walk opposite left thrice".

In [31]:
# Example 3: compositional generalization (SCAN)

# Least to Most, second step - Mapping
# Map this simplified sequence of steps into actual actions
least_to_most_second_step_prompt = """
Q: turn left
A: "turn left" outputs "TURN LEFT".

Q: turn right
A: "turn right" outputs "TURN RIGHT".

Q: jump left
A: The output of "jump left" concatenates: the output of "turn left", the output of "jump". "turn left" outputs "TURN LEFT". "jump" outputs "JUMP". So concatenating the output of "turn left" and the out- put of "jump" leads to "TURN LEFT" + "JUMP". So the output of "jump left" is "TURN LEFT" + "JUMP".

Q: run right
A: The output of "run right" concatenates: the output of "turn right", the output of "run". "turn right" outputs "TURN RIGHT". "run" outputs "RUN". So concatenating the output of "turn right" and the output of "run" leads to "TURN RIGHT" + "RUN". So the output of "run right" is "TURN RIGHT" + "RUN".

Q: look twice
A: The output of "look twice" concatenates: the output of "look", the output of "look". "look" outputs "LOOK". So repeating the output of "look" two times leads to "LOOK" * 2. So the output of "look twice" is "LOOK" * 2.

Q: run and look twice
A: The output of "run and look twice" concatenates: the output of "run", the output of "look twice". "run" outputs "RUN". "look twice" outputs "LOOK" * 2. So concatenating the output of "run" and the output of "look twice" leads to "RUN" + "LOOK" * 2. So the output of "run and look twice" is "RUN" + "LOOK" * 2.

Q: jump right thrice
A: The output of "jump right thrice" concatenates: the output of "jump right", the output of "jump right", the output of "jump right". "jump right" outputs "TURN RIGHT" + "JUMP". So repeating the output of "jump right" three times leads to ("TURN RIGHT" + "JUMP") * 3. So the output of "jump right thrice" is ("TURN RIGHT" + "JUMP") * 3.

Q: walk after run
A: The output of "walk after run" concatenates: the output of "run", the output of "walk". "run" outputs "RUN". "walk" outputs "WALK". So concatenating the output of "run" and the output of "walk" leads to "RUN" + "WALK". So the output of "walk after run" is "RUN" + "WALK".

Q: turn opposite left
A: The output of "turn opposite left" concatenates: the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" twice leads to "TURN LEFT" * 2. So the output of "turn opposite left" is "TURN LEFT" * 2.

Q: turn around left
A: The output of "turn around left" concatenates: the output of "turn left", the output of "turn left", the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" four times leads to "TURN LEFT" * 4. So the output of "turn around left" is "TURN LEFT" * 4.

Q: turn opposite right
A: The output of "turn opposite right" concatenates: the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" twice leads to "TURN RIGHT" * 2. So the output of "turn opposite right" is "TURN RIGHT" * 2.

Q: turn around right
A: The output of "turn around right" concatenates: the output of "turn right", the output of "turn right", the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" four times leads to "TURN RIGHT" * 4. So the output of "turn around right" is "TURN RIGHT" * 4.

Q: walk opposite left
A: The output of "walk opposite left" concatenates: the output of "turn opposite left", the output of "walk". "turn opposite left" outputs "TURN LEFT" * 2. "walk" outputs "WALK". So concatenating the output of "turn opposite left" and the output of "walk" leads to "TURN LEFT" * 2 + "WALK". So the output of "walk opposite left" is "TURN LEFT" * 2 + "WALK".

Q: walk around left
A: The output of "walk around left" concatenates: the output of "walk left", the output of "walk left", the output of "walk left", the output of "walk left". "walk left" outputs "TURN LEFT" + "WALK". So repeating the output of "walk around left" four times leads to ("TURN LEFT" + "WALK") * 4. So the output of "walk around left" is ("TURN LEFT" + "WALK") * 4.

Q: "jump around left twice after walk opposite left thrice" 
A:
"""

call_GPT(least_to_most_second_step_prompt, 'text-davinci-003')

Jacob: 
Q: turn left
A: "turn left" outputs "TURN LEFT".

Q: turn right
A: "turn right" outputs "TURN RIGHT".

Q: jump left
A: The output of "jump left" concatenates: the output of "turn left", the output of "jump". "turn left" outputs "TURN LEFT". "jump" outputs "JUMP". So concatenating the output of "turn left" and the out- put of "jump" leads to "TURN LEFT" + "JUMP". So the output of "jump left" is "TURN LEFT" + "JUMP".

Q: run right
A: The output of "run right" concatenates: the output of "turn right", the output of "run". "turn right" outputs "TURN RIGHT". "run" outputs "RUN". So concatenating the output of "turn right" and the output of "run" leads to "TURN RIGHT" + "RUN". So the output of "run right" is "TURN RIGHT" + "RUN".

Q: look twice
A: The output of "look twice" concatenates: the output of "look", the output of "look". "look" outputs "LOOK". So repeating the output of "look" two times leads to "LOOK" * 2. So the output of "look twice" is "LOOK" * 2.

Q: run and look twice
A: The output of "run and look twice" concatenates: the output of "run", the output of "look twice". "run" outputs "RUN". "look twice" outputs "LOOK" * 2. So concatenating the output of "run" and the output of "look twice" leads to "RUN" + "LOOK" * 2. So the output of "run and look twice" is "RUN" + "LOOK" * 2.

Q: jump right thrice
A: The output of "jump right thrice" concatenates: the output of "jump right", the output of "jump right", the output of "jump right". "jump right" outputs "TURN RIGHT" + "JUMP". So repeating the output of "jump right" three times leads to ("TURN RIGHT" + "JUMP") * 3. So the output of "jump right thrice" is ("TURN RIGHT" + "JUMP") * 3.

Q: walk after run
A: The output of "walk after run" concatenates: the output of "run", the output of "walk". "run" outputs "RUN". "walk" outputs "WALK". So concatenating the output of "run" and the output of "walk" leads to "RUN" + "WALK". So the output of "walk after run" is "RUN" + "WALK".

Q: turn opposite left
A: The output of "turn opposite left" concatenates: the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" twice leads to "TURN LEFT" * 2. So the output of "turn opposite left" is "TURN LEFT" * 2.

Q: turn around left
A: The output of "turn around left" concatenates: the output of "turn left", the output of "turn left", the output of "turn left", the output of "turn left". "turn left" outputs "TURN LEFT". So repeating the output of "turn left" four times leads to "TURN LEFT" * 4. So the output of "turn around left" is "TURN LEFT" * 4.

Q: turn opposite right
A: The output of "turn opposite right" concatenates: the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" twice leads to "TURN RIGHT" * 2. So the output of "turn opposite right" is "TURN RIGHT" * 2.

Q: turn around right
A: The output of "turn around right" concatenates: the output of "turn right", the output of "turn right", the output of "turn right", the output of "turn right". "turn right" outputs "TURN RIGHT". So repeating the output of "turn right" four times leads to "TURN RIGHT" * 4. So the output of "turn around right" is "TURN RIGHT" * 4.

Q: walk opposite left
A: The output of "walk opposite left" concatenates: the output of "turn opposite left", the output of "walk". "turn opposite left" outputs "TURN LEFT" * 2. "walk" outputs "WALK". So concatenating the output of "turn opposite left" and the output of "walk" leads to "TURN LEFT" * 2 + "WALK". So the output of "walk opposite left" is "TURN LEFT" * 2 + "WALK".

Q: walk around left
A: The output of "walk around left" concatenates: the output of "walk left", the output of "walk left", the output of "walk left", the output of "walk left". "walk left" outputs "TURN LEFT" + "WALK". So repeating the output of "walk around left" four times leads to ("TURN LEFT" + "WALK") * 4. So the output of "walk around left" is ("TURN LEFT" + "WALK") * 4.

Q: "jump around left twice after walk opposite left thrice" 
A:


GPT: 
The output of "jump around left twice after walk opposite left thrice" concatenates: the output of "walk opposite left thrice", the output of "jump around left twice". "walk opposite left thrice" outputs "(TURN LEFT" * 2 + "WALK") * 3. "jump around left twice" outputs ("TURN LEFT" + "JUMP") * 4. So concatenating the output of "walk opposite left thrice" and the output of "jump around left twice" leads to "(TURN LEFT" * 2 + "WALK") * 3 + ("TURN LEFT" + "JUMP") * 4. So the output of "jump around left twice after walk opposite left thrice" is "(TURN LEFT" * 2 + "WALK") * 3 + ("TURN LEFT" + "JUMP") * 4.

## What's in a Prompt?

Some general advice about what is important in a prompt:

- Surprsingly, the answers in the examplars of a few shot prompt are not very important.
- Although the answers themselves may not be of upmost importance, having a representative labelspace of the total population is!
- Perhaps the most important part of few shot examplars is the format!
- Between 4-8 exemplars is a good amount to use, but the more the better given the constraints of the model context window.